In [1]:
import pandas as pd
import numpy as np

In [2]:
path = r'T:\MPO\TIP\TIP FY24-27\Maps\Data'

In [3]:
file = path + '\\MTIP24_27_List.xlsx'

In [4]:
data = pd.read_excel(file)

In [5]:
data.columns

Index(['Geo', 'Project Name', 'MTIP ID #', 'Project Description', 'Work Type',
       'Perf. Meas.', 'RTP Project Number / Ref.', 'In AQ CATS?', 'In\nAQMA?',
       'Air Quality Status', 'STIP Key', 'FFY', 'Phase', 'Phase Status',
       'Federal Funding ', 'Federal Funding Source', 'Federal Req. Match',
       'Federal Req. Match Source', 'Total Fed+ Req Match', 'Other Funding',
       'Other Funding Source', 'Total All Sources'],
      dtype='object')

In [39]:
data['Air Quality Status'].unique()

array(['EXEMPT / Other-Planning and Technical Studies (IAC conf 4/26/20)',
       nan, 'EXEMPT / Other-Planning and Technical Studies',
       'N/A (IAC conf 4/26/20)', 'N/A (IAC conf 11/3/21)',
       'EXEMPT / Air Quality - Bicycle and Pedestrian facilities',
       'Outside PM10 air quality maintenance area',
       'Outside PM10 air quality maintenance area (IAC conf 11/3/21)',
       'EXEMPT / Safety - Highway Safety Improvement Program implementation',
       'EXEMPT / Other - Specific activities which do not involve or lead directly to construction',
       'EXEMPT / Other - Planning and Technical Studies',
       'EXEMPT / Safety - Projects that correct, improve, or eliminate a hazardous location or feature',
       'Part of K16223, project level conformity was approved in interagency meeting 12/23/2020',
       'EXEMPT / Safety - Highway Safety Improvement Program implementation; Air Quality - Bicycle and Pedestrian facilities (IAC conf 4/26/20)',
       'EXEMPT / Safety - Pro

In [53]:
def getAQvars(x, var=['AQ Exempt?', 'AQ Status', 'IAC']):
    if str(x) == 'nan':
        res = None
    elif var == 'AQ Exempt?':
        if 'EXEMPT' in x:
            res = 'Yes'
        else:
            res = 'No'
    
    elif var == 'IAC':
        if 'IAC' in x:
            res = 'IAC ' + x.split(' (IAC ')[1].split(')')[0]
        else:
            res = 'N/A'
    
    elif var == 'AQ Status':
        if 'EXEMPT' in x:
            if 'IAC' in x:
                res = x.split('EXEMPT / ')[1].split(' (IAC')[0]
            else:
                res = x.split('EXEMPT / ')[1]
        elif 'IAC' in x:
            res = x.split(' (IAC')[0]
        else:
            res = x
    
    return res

In [54]:
data['AQ Exempt?'] = data['Air Quality Status'].apply(lambda x: getAQvars(x, var='AQ Exempt?')) 

In [55]:
data['AQ Status'] = data['Air Quality Status'].apply(lambda x: getAQvars(x, var='AQ Status')) 

In [56]:
data['IAC'] = data['Air Quality Status'].apply(lambda x: getAQvars(x, var='IAC')) 

In [57]:
data['AQ Status'].unique()

array(['Other-Planning and Technical Studies', None, 'N/A',
       'Air Quality - Bicycle and Pedestrian facilities',
       'Outside PM10 air quality maintenance area',
       'Safety - Highway Safety Improvement Program implementation',
       'Other - Specific activities which do not involve or lead directly to construction',
       'Other - Planning and Technical Studies',
       'Safety - Projects that correct, improve, or eliminate a hazardous location or feature',
       'Part of K16223, project level conformity was approved in interagency meeting 12/23/2020',
       'Safety - Highway Safety Improvement Program implementation; Air Quality - Bicycle and Pedestrian facilities',
       'Safety - Projects that correct, improve, or eliminate a hazardous location or feature; Air Quality - Bicycle and pedestrian facilities',
       'Safety - Projects that correct, improve, or eliminate a hazardous location or feature.',
       'Air Quality - Bicycle and Pedestrian facilities; Safety - 